In [ ]:
from Declare4Py.D4PyEventLog import D4PyEventLog
from Declare4Py.ProcessMiningTasks.Discovery.DeclareMiner import DeclareMiner 
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from pybeamline.sources import log_source
from Declare4Py.ProcessMiningTasks.ConformanceChecking.MPDeclareAnalyzer import MPDeclareAnalyzer
from pybeamline.mappers.sliding_window_to_log import list_to_log
from pybeamline_declare4py_bridge.conformance.pybeamline_declare4py_conformance_checker import pybeamline_to_declare4py_log
import os
import warnings
warnings.filterwarnings("ignore")

Nessesary to mine model from Log, since Declare4py read from file is broken

In [33]:
log_path = os.path.join("../data/extension_log/extension-log-4.xes")
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log(log_path)
extension_log_model = DeclareMiner(log=event_log, consider_vacuity=False, min_support=0.2, itemsets_support=0.9, max_declare_cardinality=3).run()


parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 14091.02it/s]


Computing discovery ...


In [34]:
log_list = []
log_source("../data/extension_log/extension-log-noisy-4.xes").subscribe(lambda item: log_list.append(item))
declare_log = pybeamline_to_declare4py_log(list_to_log(log_list))


parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 14154.02it/s]


In [38]:
conformance_result = MPDeclareAnalyzer(
                log=declare_log, 
                declare_model=extension_log_model,
                consider_vacuity=False
                ).run()

result = conformance_result.get_metric(metric='state')

In [43]:
df = result.filter(regex=r"(?<!.)Response\[.*\]\ \|\ \||(?<!.)Alternate Response\[.*\]\ \|\ \||(?<!.)Chain Response\[.*\]\ \|\ \|", axis=1)
num_ones = (df == 1).sum().sum()  # Count all 1's in the dataframe
num_zeros = (df == 0).sum().sum()  # Count all 0's in the dataframe
print(f"Fulfillments: {num_ones} Violations: {num_zeros}")

Fulfillments: 6571 Violations: 429
